In [5]:
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
data = pd.read_csv('dating_app_dataset.csv')
data.head()

,User ID,Age,Gender,Height,Interests,Looking For,Children,Education Level,Occupation,Swiping History,Frequency of Usage
0,1,30,Male,5.240385,"['Sports', 'Cooking', 'Hiking', 'Music', 'Movi...",Casual Dating,No,High School,Student,96,Weekly
1,2,27,Female,4.937625,"['Sports', 'Reading']",Friendship,Yes,Master's Degree,Artist,96,Monthly
2,3,29,Female,5.806296,['Sports'],Casual Dating,No,Bachelor's Degree,Social Media Influencer,64,Daily
3,4,29,Female,5.101402,['Reading'],Marriage,No,Ph.D.,Doctor,67,Daily
4,5,32,Male,5.986670,"['Sports', 'Hiking', 'Music', 'Movies', 'Readi...",Long-term Relationship,Yes,Ph.D.,Engineer,93,Monthly


In [8]:
# age distribution

fig = px.histogram(data, 
                   x = 'Age',
                   color = 'Gender',
                   nbins = 20,
                   title = "Age Distribution by Gender")
fig.update_layout(xaxis_title = 'Age',
                  yaxis_title = 'Count')
fig.show()

In [10]:
# education level

education_order = ["High School", "Bachelor's Degree", "Master's Degree", "Ph.D."]
fig = px.bar(data, 
             x = "Education Level",
             color = "Gender",
             category_orders = {"Education Level": education_order},
             title = 'Education Level Distribution by Gender')

fig.update_layout(xaxis_title = "Education Level",
                  yaxis_title = "Count")

fig.show()

In [13]:
fig = px.bar(data,
             x = "Frequency of Usage",
             color = "Gender",
             title = "Frequency of App Usage Distribution")
fig.update_layout(xaxis_title = "Frequency of Usage",
                  yaxis_title = "Count")
fig.show()


In [14]:
# Separate data into male and female
male_profiles = data[data['Gender'] == 'Male']
female_profiles = data[data['Gender'] == 'Female']

In [22]:
def calculate_match_score(profile1, profile2):
    interest1 = set(eval(profile1['Interests']))
    interest2 = set(eval(profile2['Interests']))
    shared_interests_score = len(interest1.intersection(interest2))
    age_difference_score = max(0, 10 - abs(profile1['Age'] - profile2['Age']))
    swiping_history_score = min(profile1['Swiping History'], profile2['Swiping History'] / 100)
    relationship_type_score = 0
    if profile1['Looking For'] == profile2['Looking For']:
        relationship_type_score = 1
    total_score = (
        shared_interests_score + age_difference_score + swiping_history_score + relationship_type_score
    )
    return total_score

profile1 = male_profiles.iloc[0]
profile2 = female_profiles.iloc[0]
match_score = calculate_match_score(profile1, profile2)
print(f"Match score between user {profile1['User ID']} and user {profile2['User ID']} is: {match_score}")


Match score between user 1 and user 2 is: 9.96


In [25]:
def recommend_profiles(male_profiles, female_profiles):
    recommendations = []

    for _, male_profile in male_profiles.iterrows():
        best_match = None
        best_score = -1

        for _, female_profile in female_profiles.iterrows():
            score = calculate_match_score(male_profile, female_profile)

            if score > best_score:
                best_match = female_profile
                best_score = score

        recommendations.append((male_profile, best_match, best_score))
    
    return recommendations

recommendations = recommend_profiles(male_profiles, female_profiles)

recommendations.sort(key = lambda x: x[2], reverse=True)

for idx, (male_profile, female_profile, match_score) in enumerate(recommendations[:10]):
    print(f"Recommendation {idx + 1}:")
    print(f"Male Profile (User {male_profile['User ID']}): Age {male_profile['Age']}, Interest {male_profile['Interests']}")
    print(f"Female Profile (User {female_profile['User ID']}): Age {female_profile['Age']}, Interest {female_profile['Interests']}")
    print(f"Match Score: {match_score}")
    print()

Recommendation 1:
Male Profile (User 456): Age 29, Interest ['Cooking', 'Hiking', 'Sports', 'Travel', 'Music', 'Movies', 'Reading']
Female Profile (User 65): Age 29, Interest ['Travel', 'Movies', 'Reading', 'Sports', 'Music', 'Cooking', 'Hiking']
Match Score: 19.0

Recommendation 2:
Male Profile (User 147): Age 34, Interest ['Reading', 'Travel', 'Movies', 'Hiking', 'Cooking', 'Music', 'Sports']
Female Profile (User 287): Age 34, Interest ['Reading', 'Hiking', 'Cooking', 'Music', 'Movies', 'Travel', 'Sports']
Match Score: 18.98

Recommendation 3:
Male Profile (User 489): Age 33, Interest ['Travel', 'Hiking', 'Reading', 'Sports', 'Music', 'Movies', 'Cooking']
Female Profile (User 99): Age 33, Interest ['Reading', 'Cooking', 'Sports', 'Hiking', 'Movies', 'Music', 'Travel']
Match Score: 18.97

Recommendation 4:
Male Profile (User 36): Age 19, Interest ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interest ['Reading', 'Music', 'Co